In [1]:
import json
import hashlib
import sqlite3
from datetime import datetime
from uuid import uuid4
from urllib.parse import urlparse
from flask import Flask, jsonify, request
import logging
import requests
import random
import string

# main_script.py
import account_manager
from Blockchain import Blockchain
from Blockchain import log_change

app = Flask(__name__)
# Initialize the SQLite database
conn = sqlite3.connect('p2p_energy_trading.db', check_same_thread=False)
cursor = conn.cursor()

# Check if the block_hash column exists, and if not, add it
def add_block_hash_column():
    cursor.execute("PRAGMA table_info(Blockchain);")
    columns = [column[1] for column in cursor.fetchall()]
    
    if 'block_hash' not in columns:
        cursor.execute("ALTER TABLE Blockchain ADD COLUMN block_hash TEXT;")
        conn.commit()
        print("Added 'block_hash' column to the Blockchain table.")

add_block_hash_column()

# Create tables for blocks, transactions, and blockchain logs
create_tables = '''
CREATE TABLE IF NOT EXISTS Blockchain (
    block_id INTEGER PRIMARY KEY AUTOINCREMENT,
    block_index INTEGER,
    timestamp TEXT,
    proof INTEGER,
    previous_hash TEXT,
    block_hash TEXT
);
'''

create_transactions_table = '''
CREATE TABLE IF NOT EXISTS Transactions (
    transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
    block_id INTEGER,
    Seller TEXT,
    Buyer TEXT,
    Power REAL,
    Price REAL
);
'''

create_logs_table = '''
CREATE TABLE IF NOT EXISTS BlockchainLogs (
    log_id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    operation_type TEXT,
    details TEXT
);
'''

cursor.execute(create_tables)
cursor.execute(create_transactions_table)
cursor.execute(create_logs_table)
conn.commit()

# Helper function to log changes in the BlockchainLogs table
def log_change(operation_type, details):
    with sqlite3.connect('p2p_energy_trading.db', timeout=10, check_same_thread=False) as conn:
        cursor = conn.cursor()
        timestamp = str(datetime.now())
        cursor.execute('''INSERT INTO BlockchainLogs (timestamp, operation_type, details) 
                          VALUES (?, ?, ?)''', 
                       (timestamp, operation_type, json.dumps(details)))
        conn.commit()
    print(f"[{timestamp}] {operation_type}: {details}")

# Function to insert a block and associated transactions into the database
def insert_block_to_db(block, transactions):
    with sqlite3.connect('p2p_energy_trading.db', timeout=10, check_same_thread=False) as conn:
        cursor = conn.cursor()
        cursor.execute('''INSERT INTO Blockchain (block_index, timestamp, proof, previous_hash, block_hash) 
                          VALUES (?, ?, ?, ?, ?)''', 
                       (block['index'], block['timestamp'], block['proof'], block['previous_hash'], block['block_hash']))
        conn.commit()
        block_id = cursor.lastrowid
        
        for tx in transactions:
            cursor.execute('''INSERT INTO Transactions (block_id, Seller, Buyer, Power, Price) 
                              VALUES (?, ?, ?, ?, ?)''', 
                           (block_id, tx['Seller'], tx['Buyer'], tx['Power'], tx['Price']))
        conn.commit()
        return block_id
    

# Blockchain Class
class Blockchain:
    def __init__(self):
        self.chain = [] 
        self.pending_transactions = []  # Temporary store for unmatched transactions
        self.current_transactions = []  # Store matched transactions for the next block
        self.nodes = set()
        self.new_block(previous_hash='1', proof=1000)

    def new_block(self, proof, previous_hash=None):
        if previous_hash is None:
            previous_hash = self.last_block['block_hash'] if self.chain else '1' 
        
        block = {
            'index': len(self.chain) + 1,
            'timestamp': str(datetime.now()),
            'transactions': self.current_transactions,
            'proof': proof,
            'previous_hash': previous_hash,
        }
        
        block_hash = self.hash(block)
        block['block_hash'] = block_hash
        
        
        # Store block and transactions in the database
        insert_block_to_db(block, self.current_transactions) # Pass transactions to save
        
        # Log the new block creation
        log_change("Block Creation", {"block_index": block['index'], "proof": proof, "previous_hash": previous_hash})
        self.chain.append(block)
        self.current_transactions = []  # Clear matched transactions for the next block
        
        # Validate blockchain integrity
        if not self.validate_chain():
            print("Blockchain integrity check failed!")
            log_change("Blockchain Failure", {"reason": "Hash mismatch"})
        return block
    
        # Define add_transaction here
    def add_transaction(self, sender, receiver, power, price, role):
        transaction = {
            "sender": sender,
            "receiver": receiver,
            "power": power,
            "price": price,
            "role": role  # Role: "seller" or "buyer"
        }
        self.pending_transactions.append(transaction)
        print(f"Transaction added to pending: {transaction}")
         # Attempt to match transactions
        self.match_transactions()
        
        
    def match_transactions(self):
        sellers = [t for t in self.pending_transactions if t["role"] == "seller"]
        buyers = [t for t in self.pending_transactions if t["role"] == "buyer"]

        for seller in sellers:
            match_found = False
            for buyer in buyers:
                # Check for matching conditions: sender and receiver IDs in reverse, and equal power
                if (seller["sender"] == buyer["receiver"] and
                    seller["receiver"] == buyer["sender"] and
                    seller["power"] == buyer["power"]):

                    # Matched transaction to store in current_transactions
                    matched_transaction = {
                        "Seller": seller["sender"],
                        "Buyer": buyer["sender"],
                        "Power": seller["power"],
                        "Price": (float(seller["price"]) + float(buyer["price"])) / 2  # Convert prices to float and average
                    }
                    self.current_transactions.append(matched_transaction)

                    # Remove matched transactions from pending
                    self.pending_transactions.remove(seller)
                    self.pending_transactions.remove(buyer)
                    print(f"Matched transaction added: {matched_transaction}")
                    match_found = True
                    break  # Exit inner loop after finding a match for the seller

                if not match_found:
                    print(f"No match found for transaction: {seller}")
                    
        return self.last_block['index'] + 1
        
    def insert_block_to_db(self, block, transactions):
        cursor.execute('''
            INSERT INTO Blockchain (block_index, timestamp, proof, previous_hash, block_hash) 
            VALUES (?, ?, ?, ?, ?)
        ''', (block['index'], block['timestamp'], block['proof'], block['previous_hash'], block['block_hash']))
        conn.commit()

        block_id = cursor.lastrowid
        for tx in transactions:
            cursor.execute('''
                INSERT INTO Transactions (block_id, Seller, Buyer, Power, Price)
                VALUES (?, ?, ?, ?, ?)
            ''', (block_id, tx['Seller'], tx['Buyer'], tx['Power'], tx['Price']))
        conn.commit()
    
    def validate_chain(self):
        # Validate the blockchain by checking hash links between blocks
        for i in range(1, len(self.chain)):
            previous_block = self.chain[i - 1]
            current_block = self.chain[i]
            #current_block['previous_hash'] = previous_block['block_hash']
            if current_block['previous_hash'] != previous_block['block_hash']:
                print(f"Validation failed: {current_block['previous_hash']} != {previous_block['block_hash']}")
                return False
            
            if not self.valid_proof(previous_block['proof'], current_block['proof']):
                print(f"Proof of work validation failed for block {current_block['index']}")
                return False

        return True
    
    @property
    def last_block(self):
        # Returns the last block in the chain
        return self.chain[-1]
    @staticmethod
    def hash(block):
        # Hash a block to generate a unique identifier
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()
   
    @staticmethod
    def valid_proof(last_proof, proof):
        # Check if the proof of work is valid
        this_proof = f'{proof}{last_proof}'.encode()
        this_proof_hash = hashlib.sha256(this_proof).hexdigest()
        return this_proof_hash[:4] == '0000'

    def proof_of_work(self, last_proof):
        # Proof of work algorithm
        proof = 0
        while not self.valid_proof(last_proof, proof):
            proof += 1
        return proof
    
    def register_node(self, address):
        parsed_url = urlparse(address)
        self.nodes.add(parsed_url.netloc)
    
    def resolve_conflicts(self):
        neighbours = self.nodes.copy()
        max_length = len(self.chain)
        new_chain = None
        
        for node in neighbours:
            try:
                response = requests.get(f'http://{node}/chain').json()
                length = response['length']
                chain = response['chain']
                if length > max_length and self.valid_chain(chain):
                    max_length = length
                    new_chain = chain
            except requests.exceptions.RequestException as e:
                print(f"Error connecting to node {node}: {e}")

        if new_chain:
            self.chain = new_chain
            log_change("Chain Replaced", {"new_length": len(self.chain)})
            return True
        return False


node_id = str(uuid4())
blockchain = Blockchain()

@app.route('/add_account', methods=['POST'])
def add_account():
    try:
        # Get JSON data from request
        data = request.json
        
        # Validate required fields
        required_fields = ['name']
        if not all(field in data for field in required_fields):
            return jsonify({"error": "Missing required fields"}), 400
        
        # Extract name from JSON data
        name = data['name'].strip()
        
        # Create the account
        new_account = account_manager.create_account(name)
        
        if new_account is None:
            raise ValueError("Failed to create account")
        
        # Log successful creation
        log_change("Account Created", {"account_id": new_account['id'], "name": name})
        
        return jsonify({
            "message": "Account created successfully",
            "account_id": new_account['id'],
            "public_key": new_account['public_key']
        }), 201
    
    except sqlite3.Error as e:
        logging.error(f"Database error: {e}")
        return jsonify({"error": f"Database error occurred"}), 500
    
    except ValueError as ve:
        logging.error(f"Value error: {ve}")
        return jsonify({"error": str(ve)}), 400
    
    except Exception as e:
        logging.error(f"Unexpected error: {str(e)}\nTraceback:\n{traceback.format_exc()}")
        return jsonify({"error": "An unexpected error occurred"}), 500

@app.route('/mine')
def mine():
    try:
        last_block = blockchain.last_block
        last_proof = last_block['proof']
        proof = blockchain.proof_of_work(last_proof)
        
        # Calculate the hash of the last block's data
        last_block_data = {
            'index': last_block['index'],
            'timestamp': last_block['timestamp'],
            'transactions': last_block['transactions'],
            'proof': last_block['proof'],
            'previous_hash': last_block['previous_hash']
        }
        previous_hash = blockchain.hash(last_block_data)
        block = blockchain.new_block(proof, previous_hash)
        
        response = {
            'message': 'New block created',
            'index': block['index'],
            'transactions': block['transactions'],
            'proof': block['proof'],
            'previous_hash': block['previous_hash'],
            'block_hash': block['block_hash']
        }
        return jsonify(response), 200
    except Exception as e:
        logging.error(f"Error mining block: {str(e)}")
        return jsonify({'error': str(e)}), 500

@app.route('/add_transaction', methods=['POST'])
def add_transaction():
    values = request.json
    required = ["sender", "receiver", "power", "price", "role"]
    if not all(k in values for k in required):
        return "Missing values", 400 
    
    if values["role"] == "seller":
        blockchain.add_transaction(values["sender"], values["receiver"], values["power"], values["price"], "seller")
    elif values["role"] == "buyer":
        blockchain.add_transaction(values["sender"], values["receiver"], values["power"], values["price"], "buyer")
    return jsonify({"message": "Transaction will be processed"}), 201

@app.route('/chain')
def full_chain():
    response = {
        'chain': blockchain.chain,
        'length': len(blockchain.chain),
    }
    return jsonify(response), 200

@app.route('/nodes/register', methods=['POST'])
def register_node():
    values = request.get_json()
    nodes = values.get('nodes')
    if nodes is None:
        return "Error: Please supply a valid list of nodes", 400 

    for node in nodes:
        blockchain.register_node(node)
        
    response = {
        'message': 'New nodes have been added',
        'total_nodes': list(blockchain.nodes)
    }
    return jsonify(response), 201

@app.route('/nodes/resolve')
def consensus():
    replaced = blockchain.resolve_conflicts()
    if replaced:
        response = {
            'message': 'Our chain was replaced',
            'new_chain': blockchain.chain
        }
    else:
        response = {
            'message': 'Our chain is authoritative',
            'new_chain': blockchain.chain
        }
    return jsonify(response), 200

if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO)
    app.run(host='0.0.0.0', port=6000)

OperationalError: no such table: Blockchain

In [2]:
import json
import hashlib
from flask import Flask, request, jsonify


In [ ]:
app = Flask(__name__)
class Blockchain:
    def __init__(self):
        self.chain = [] 
        self.pending_transactions = []  # Temporary store for unmatched transaction
        self.current_transactions = []  # Store matched transactions for the next block
        
    def add_transaction(self, sender, receiver, power, price, role):
        transaction = {
            "sender": sender,
            "receiver": receiver,
            "power": power,
            "price": price,
            "role": role  # Role: "seller" or "buyer"
        }
        self.pending_transactions.append(transaction)
        print(f"Transaction added to pending: {transaction}")
         # Attempt to match transactions
        self.match_transactions()


    def match_transactions(self):
        sellers = [t for t in self.pending_transactions if t["role"] == "seller"]
        buyers = [t for t in self.pending_transactions if t["role"] == "buyer"]

        for seller in sellers:
            match_found = False
            for buyer in buyers:
                # Check for matching conditions: sender and receiver IDs in reverse, and equal power
                if (seller["sender"] == buyer["receiver"] and
                    seller["receiver"] == buyer["sender"] and
                    seller["power"] == buyer["power"]):

                    # Matched transaction to store in current_transactions
                    matched_transaction = {
                        "Seller": seller["sender"],
                        "Buyer": buyer["sender"],
                        "Power": seller["power"],
                        "Price": (float(seller["price"]) + float(buyer["price"])) / 2  # Convert prices to float and average
                    }
                    self.current_transactions.append(matched_transaction)

                    # Remove matched transactions from pending
                    self.pending_transactions.remove(seller)
                    self.pending_transactions.remove(buyer)
                    print(f"Matched transaction added: {matched_transaction}")
                    match_found = True
                    break  # Exit inner loop after finding a match for the seller

                if not match_found:
                    print(f"No match found for transaction: {seller}")
    
blockchain = Blockchain()

@app.route('/add_transaction', methods=['POST'])
def add_transaction():
    values = request.json
    required = ["sender", "receiver", "power", "price", "role"]
    if not all(k in values for k in required):
        return "Missing values", 400
         
    if values["role"] == "seller":
        blockchain.add_transaction(values["sender"], values["receiver"], values["power"], values["price"], "seller")
    elif values["role"] == "buyer":
        blockchain.add_transaction(values["sender"], values["receiver"], values["power"], values["price"], "buyer")
    return jsonify({"message": "Transaction will be processed"}), 201
    
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=6000)    

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:6000
 * Running on http://192.168.1.102:6000
Press CTRL+C to quit
